In [1]:
import numpy as np 
import pandas as pd
import json 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score

from tqdm import tqdm
tqdm.pandas()
from sklearn.linear_model import ElasticNet
import statsmodels.api as sm
from copy import deepcopy

import Utils
import models

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_json('./FeatureImportance_CLS.json')

In [3]:
df = df.transpose()

In [4]:
df

,0,1
O_12,"[[O_SUB, 0.323851591312983], [SUB2, 0.10272724...",RF
O_23,"[[SUB3, 0.30051052570343], [DIFF_SUB32, 0.1838...",GB
O_ADD,"[[O_SUB, 0.288698980033186], [SUB3, 0.18133476...",RF
O_DIV,"[[O_SUB, 0.296814166231604], [SUB3, 0.19470369...",RF
O_MUL,"[[O_SUB, 0.43333867192268305], [SUB3, 0.295425...",GB
O_SUB,"[[O_MUL, 0.13120858371257701], [MUL2, 0.089517...",GB


In [5]:
cols = []

for idx in df.index:
    cur_names = df[0][idx]
    ls_names = [name[0] for name in cur_names]

    cols.append(ls_names)

In [6]:
cols

[['O_SUB', 'SUB2', 'n_sum', 'ADD2', 'ADD1'],
 ['SUB3', 'DIFF_SUB32', 'O_SUB', 'SUB2', 'DIFF_DIV21'],
 ['O_SUB', 'SUB3', 'SUB2', 'SUB1', 'MUL1'],
 ['O_SUB', 'SUB3', 'SUB2', 'ADD2', 'SUB1'],
 ['O_SUB', 'SUB3', 'ADD2', 'DIFF_ACC_SUB32', 'ADD1'],
 ['O_MUL', 'MUL2', 'DIFF_MUL31', 'ADD2', 'DIV2']]

In [7]:
for j in range(5):
    name = 'feature ' + str(j+1)
    df[name] = np.array(cols)[:,j]
    

In [8]:
df

,0,1,feature 1,feature 2,feature 3,feature 4,feature 5
O_12,"[[O_SUB, 0.323851591312983], [SUB2, 0.10272724...",RF,O_SUB,SUB2,n_sum,ADD2,ADD1
O_23,"[[SUB3, 0.30051052570343], [DIFF_SUB32, 0.1838...",GB,SUB3,DIFF_SUB32,O_SUB,SUB2,DIFF_DIV21
O_ADD,"[[O_SUB, 0.288698980033186], [SUB3, 0.18133476...",RF,O_SUB,SUB3,SUB2,SUB1,MUL1
O_DIV,"[[O_SUB, 0.296814166231604], [SUB3, 0.19470369...",RF,O_SUB,SUB3,SUB2,ADD2,SUB1
O_MUL,"[[O_SUB, 0.43333867192268305], [SUB3, 0.295425...",GB,O_SUB,SUB3,ADD2,DIFF_ACC_SUB32,ADD1
O_SUB,"[[O_MUL, 0.13120858371257701], [MUL2, 0.089517...",GB,O_MUL,MUL2,DIFF_MUL31,ADD2,DIV2


In [40]:
df.drop(columns=[0]).to_excel('FI_CLS.xlsx')

In [9]:
df = pd.read_excel('Cleaned_data.xlsx')
df1 = pd.read_excel('PMT_N+2SD+3SD.xlsx', sheet_name='N-2SD')


In [10]:
df

,age,gender,log,ADD1,ADD2,ADD3,DIV1,DIV2,DIV3,MUL1,...,rt_2_cl,rt_3_cl,rt_4_cl,rt_5_cl,rt_6_cl,n_sum,rt_mean,rt_cmt_mean,rt_bal,rt_cl
0,21,F,P-PSH-21-F-CC-9999-MOON2021-DRXX-0017,0.857143,1.000000,1.000000,0.571429,0.222222,0.750000,0.777778,...,1.280286,1.525667,1.341479,1.661812,1.733108,200,8.374756,1.424394,1.406735,1.442054
1,22,M,P-AKR-22-M-CC-9999-MOON2021-FZXX-0020,0.923077,0.750000,0.857143,0.909091,0.642857,0.225490,1.000000,...,1.175483,1.383899,1.505657,1.555550,1.596242,282,3.584204,1.281989,1.165923,1.398056
2,26,F,P-VTE-26-F-CC-9999-MOON2021-AVXX-0021,0.901639,0.960000,0.700000,0.758621,0.923077,0.833333,0.967213,...,1.332060,1.531029,1.695992,1.855483,1.838667,220,4.714379,1.336453,1.140416,1.532491
3,21,F,P-VVN-21-F-CC-9999-MOON2021-NBXX-0022,0.857143,0.545455,0.625000,0.789474,0.000000,0.363636,0.962963,...,1.405860,1.515236,1.716597,1.810916,1.720678,167,8.011617,1.407329,1.196113,1.618546
4,20,F,P-PKI-20-F-CC-9999-MOON2021-NDXX-0023,0.878788,0.515152,0.380952,0.735294,0.483871,0.421053,0.805556,...,1.039899,1.152940,1.595164,1.322001,1.289122,224,3.474093,1.251194,1.202659,1.299728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,13,F,P-VMU-13-F-7A-0006-VOON2022-VSXX-2448,0.944444,0.941176,0.666667,0.794118,0.454545,0.285714,0.897436,...,1.042415,1.454189,1.425826,1.499109,1.407632,183,5.609418,1.243259,1.201607,1.284911
216,14,M,P-PMO-14-M-8A-RZ25-VION2022-EBXX-2457,0.957000,0.905000,0.500000,0.933000,0.941000,0.833000,0.962000,...,1.521549,1.975812,1.888273,2.078682,1.959939,229,3.002000,1.688125,1.608419,1.767831
217,17,F,P-SKU-17-F-11B-1210-MOON2022-SGXX-2458,0.857000,0.545000,0.444000,0.962000,0.841000,0.894000,0.897000,...,0.963200,1.237517,1.571598,1.554479,1.590773,170,3.922667,1.179426,1.068871,1.289982
218,14,M,P-MMI-14-M-7C-1874-MOON2022-SGXX-2460,0.973000,0.923000,0.800000,1.000000,0.921000,0.870000,0.930000,...,1.353411,1.382298,1.414847,1.841210,1.773672,211,3.746583,1.379223,1.304954,1.453492


In [11]:
over_perf = df1.filter(regex = 'Overperformers')
over_perf['log'] = df1['LOG']
#over_perf = over_perf.drop(columns=['Overperformers on levels 1 and 2', 'Overperformers on levels 2 and 3'])
over_perf = over_perf.fillna(0).loc[:342]
df = df.merge(over_perf, on='log', how='left')


df = df.drop(columns=['log'])
df['gender'] = pd.factorize(df['gender'])[0]


df.columns = ['age', 'gender', 'ACC_ADD1', 'ACC_ADD2', 'ACC_ADD3', 'ACC_DIV1', 'ACC_DIV2', 'ACC_DIV3', 'ACC_MUL1',
       'ACC_MUL2', 'ACC_MUL3', 'ACC_SUB1', 'ACC_SUB2', 'ACC_SUB3', 'RT_ADD1', 'RT_ADD2', 'RT_ADD3',
       'RT_DIV1', 'RT_DIV2', 'RT_DIV3', 'RT_MUL1', 'RT_MUL2', 'RT_MUL3',
       'RT_SUB1', 'RT_SUB2', 'RT_SUB3', 'ADD1', 'ADD2', 'ADD3', 'DIV1',
       'DIV2', 'DIV3', 'MUL1', 'MUL2', 'MUL3', 'SUB1', 'SUB2',
       'SUB3', 'm_score_bal', 'acc_1_bal', 'acc_2_bal', 'acc_3_bal',
       'acc_4_bal', 'acc_5_bal', 'acc_6_bal', 'rt_1_bal', 'rt_2_bal',
       'rt_3_bal', 'rt_4_bal', 'rt_5_bal', 'rt_6_bal', 'm_score_cl',
       'acc_1_cl', 'acc_2_cl', 'acc_3_cl', 'acc_4_cl', 'acc_5_cl', 'acc_6_cl',
       'rt_1_cl', 'rt_2_cl', 'rt_3_cl', 'rt_4_cl', 'rt_5_cl', 'rt_6_cl',
       'n_sum', 'rt_mean', 'rt_cmt_mean', 'rt_bal', 'rt_cl', 'O_12', 'O_23', 'O_ADD', 'O_DIV','O_MUL', 'O_SUB']


df['O_12'][(df['O_12'] == ' 1+2')] = 1
df['O_12'] = df['O_12'].astype(int)
df['O_23'][(df['O_23'] == ' 2+3')] = 1
df['O_23'] = df['O_23'].astype(int)


seed = 0xAB0BA
np.random.seed(seed)

In [12]:
types = ['ACC_', 'RT_', '']
names = ['ADD', "DIV", "MUL", "SUB"]

for t in types:
    for name in names:
        col_name = t+name
        new_name = 'DIFF_' + col_name

        df[new_name + '21'] = df[col_name+'2'] - df[col_name+'1']
        df[new_name + '32'] = df[col_name+'3'] - df[col_name+'2']
        df[new_name + '31'] = df[col_name+'3'] - df[col_name+'1']



In [13]:
def create_y(df, cls=False):
    var_names = []

    if not cls:

        for i in range(26, 38):
            name = df.iloc[:,i].name
            vals = df.iloc[:,i]

            var_names.append(name)

            globals()[name] = vals

        for i in range(2, 14):
            name = df.iloc[:,i].name
            vals = df.iloc[:,i]

            var_names.append(name)

            globals()[name] = vals


        global ADD 
        global DIV
        global SUB
        global MUL

        global SUM
        
        ADD = ADD1 + ADD2 + ADD3
        DIV = DIV1 + DIV2 + DIV3
        SUB = SUB1 + SUB2 + SUB3
        MUL = MUL1 + MUL2 + MUL3
        
        SUM = ADD + DIV + SUB + MUL

        var_names += ['ADD', 'DIV', "MUL", 'SUB', "SUM"]

    else:
        for i in range(69, 75):
            name = df.iloc[:,i].name
            vals = df.iloc[:,i]

            var_names.append(name)

            globals()[name] = vals

    
    return var_names







var_names = create_y(df)

var_names_cls = create_y(df, cls=True)

# Statmodels

In [11]:
results = {}

for name in tqdm(var_names):
    
    y = globals()[name]
    
    X = Utils.make_X(df, name)
    
    y_pred, y_test, r2, rmse = models.LinRegSklearn(X, y, alpha = 3.15, l1_ratio = 0.1, normalize = False, max_iter = 5000)
    
    results[name] = rmse

100%|██████████| 29/29 [00:00<00:00, 45.26it/s]


In [12]:
results

{'ADD1': 6.636731369335789,
 'ADD2': 3.272932812384801,
 'ADD3': 2.466449449149984,
 'DIV1': 6.6354430041517265,
 'DIV2': 3.811576571420508,
 'DIV3': 1.9540534093153807,
 'MUL1': 6.598982412791593,
 'MUL2': 2.7774286020580616,
 'MUL3': 2.1962319232915632,
 'SUB1': 6.232658490166705,
 'SUB2': 2.8833083690343484,
 'SUB3': 1.6607113476959185,
 'ACC_ADD1': 0.10553090292237144,
 'ACC_ADD2': 0.16961356591505222,
 'ACC_ADD3': 0.21610031050299014,
 'ACC_DIV1': 0.16363677020316802,
 'ACC_DIV2': 0.24851286131851277,
 'ACC_DIV3': 0.2605392790692473,
 'ACC_MUL1': 0.13646243744789482,
 'ACC_MUL2': 0.2226313895020094,
 'ACC_MUL3': 0.27579770173212576,
 'ACC_SUB1': 0.13047999610681327,
 'ACC_SUB2': 0.25053092621181466,
 'ACC_SUB3': 0.3756052659154964,
 'ADD': 6.88506387876382,
 'DIV': 10.48971297714124,
 'MUL': 8.97036658439335,
 'SUB': 7.738309178043247,
 'SUM': 1.1320211018096382}

In [13]:
results = {}

for name in tqdm(var_names_cls):
    
    y = globals()[name]
    
    X = Utils.make_X(df, name)
    
    y_pred, y_test, metrics = models.LogRegSklearn(X, y, max_iter = 5000)
    
    roc_auc = roc_auc_score(y_test, y_pred)
    
    results[name] = (metrics[0], metrics[1], roc_auc)

100%|██████████| 6/6 [00:08<00:00,  1.49s/it]


In [14]:
results

{'O_12': (0.8582314371788056, 0.8636363636363636, 0.7142857142857143),
 'O_23': (0.9696969696969697, 0.9090909090909091, 0.9523809523809523),
 'O_ADD': (0.7612471405574854, 0.7575757575757576, 0.7549857549857549),
 'O_DIV': (0.8876033057851239, 0.8181818181818182, 0.6262295081967213),
 'O_MUL': (0.8787878787878788, 0.8787878787878788, 0.7155172413793104),
 'O_SUB': (0.7685227272727272, 0.7727272727272727, 0.6932773109243697)}

In [15]:
results = {}

for name in tqdm(var_names):
    X = Utils.make_X(df, name)
    y = globals()[name]
    
    res = models.LinRegStatmodels(X, y)
    
    r2 = res.rsquared
    params = res.params.index
    
    results[name] = res

  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [00:26<00:00,  1.09it/s]


In [16]:
params_frequency = {}
for name in df.columns:
    params_frequency[name] = set()

params_frequency['const'] = set()

for res in results:
    for param_name in results[res].params.index:
        params_frequency[param_name].add(res)

for name in params_frequency:
    new_input = str(params_frequency[name])
    if new_input == 'set()':
        new_input = ''
    params_frequency[name] = new_input

with open("params_in_models.json", "w") as outfile:
    json.dump(params_frequency, outfile, indent=4)


r2_dict = {}
for name in results:
    r2_dict[name] = results[name].rsquared

with open("r2_LinReg.json", "w") as outfile:
    json.dump(r2_dict, outfile, indent=4)


rmse_dict = {}
for name in results:
    rmse_dict[name] = float(np.sqrt(results[name].mse_total))

with open("rmse_LinReg.json", "w") as outfile:
    json.dump(rmse_dict, outfile, indent=4)

In [18]:
results['ACC_ADD1'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ACC_ADD1   R-squared:                       0.561
Model:                            OLS   Adj. R-squared:                  0.545
Method:                 Least Squares   F-statistic:                     33.77
Date:                Tue, 21 Nov 2023   Prob (F-statistic):           6.36e-34
Time:                        14:47:06   Log-Likelihood:                 311.47
No. Observations:                 220   AIC:                            -604.9
Df Residuals:                     211   BIC:                            -574.4
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.3256      0.048      6.853      0.000       0.232       0.419
ACC_DIV2           0.0443      0.022      2.023      0.044       0.001       0.087
ACC_DIV3          -0.0391      0.017     -2.273      0.024      -0.073      -0.005
ACC_MUL1           0.1772      0.036      4.862      0.000       0.105       0.249
ACC_MUL3           0.0883      0.023      3.761      0.000       0.042       0.135
ACC_SUB1           0.1770      0.026      6.737      0.000       0.125       0.229
ACC_SUB2           0.0847      0.018      4.592      0.000       0.048       0.121
ACC_SUB3           0.0745      0.016      4.777      0.000       0.044       0.105
RT_SUB1            0.0084      0.003      2.978      0.003       0.003       0.014
DIFF_ACC_MUL31    -0.0889      0.018     -5.061      0.000      -0.123      -0.054
DIFF_ACC_SUB21    -0.0923      0.019     -4.905      0.000      -0.129      -0.055
DIFF_ACC_SUB31    -0.1025      0.014     -7.530      0.000      -0.129      -0.076
==============================================================================
Omnibus:                       35.969   Durbin-Watson:                   2.179
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               97.532
Skew:                          -0.692   Prob(JB):                     6.63e-22
Kurtosis:                       5.953   Cond. No.                     1.11e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.88e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
# for name in tqdm(results):


#     plt.rc('figure', figsize=(12, 7))
#     #plt.text(0.01, 0.05, str(model.summary()), {'fontsize': 12}) old approach
#     plt.text(0.01, 0.05, str(results[name].summary()), {'fontsize': 10}, fontproperties = 'monospace') # approach improved by OP -> monospace!
#     plt.axis('off')
#     plt.tight_layout()
#     plt.savefig(name+'.jpg')
#     plt.show()


# ML

In [19]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier

## Regression

### Baseline 

In [9]:

regressors = [KNeighborsRegressor(7),
    ElasticNet(),
    BayesianRidge(),
    SVR(C=1.0, epsilon=0.2),
    GaussianProcessRegressor(),
    DecisionTreeRegressor(),
    AdaBoostRegressor(n_estimators=200),
    GradientBoostingRegressor(n_estimators=200),
    RandomForestRegressor(n_estimators=400),
    MLPRegressor()
]

In [10]:
score = {}
y = globals()['MUL']

X = Utils.make_X(df, 'MUL')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)



for model in tqdm(regressors):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score[model] = mean_squared_error(y_pred,y_test, squared=False)

100%|██████████| 10/10 [00:02<00:00,  4.64it/s]


In [11]:
score

{KNeighborsRegressor(n_neighbors=7): 13.150736392162734,
 ElasticNet(): 11.189444784832082,
 BayesianRidge(): 11.003424112523298,
 SVR(epsilon=0.2): 17.067402587239716,
 GaussianProcessRegressor(): 59.52794098216196,
 DecisionTreeRegressor(): 17.009355714376188,
 AdaBoostRegressor(n_estimators=200): 11.152850855251387,
 GradientBoostingRegressor(n_estimators=200): 10.798462575200707,
 RandomForestRegressor(n_estimators=400): 10.339087347441762,
 MLPRegressor(): 23.705053771459202}

### GB

In [20]:
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score


In [21]:
result = {}
for name in tqdm(var_names):
    y = globals()[name]
    X = Utils.make_X(df, name)

    op, rmse, model = models.XGBReg(X, y)

    result[name] = (op, rmse, model)

100%|██████████| 29/29 [05:22<00:00, 11.11s/it]


In [23]:
rmse_dic_xgb = {}
opt_params = {}
for elem in result:
    rmse_dic_xgb[elem] = result[elem][1]
    opt_params[elem] = result[elem][0]

In [24]:
with open("rmse_XGB.json", "w") as outfile:
    json.dump(rmse_dic_xgb, outfile, indent=4)
with open("params_XGB.json", "w") as outfile:
    json.dump(opt_params, outfile, indent=4)

### RF

In [26]:
result = {}
for name in tqdm(var_names):
    y = globals()[name]
    X = Utils.make_X(df, name)

    op, rmse, model = models.RFReg(X, y)

    result[name] = (op, rmse, model)

  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [55:38<00:00, 115.11s/it]


In [27]:
rmse_dic_rf = {}
opt_params = {}
for elem in result:
    rmse_dic_rf[elem] = result[elem][1]
    opt_params[elem] = result[elem][0]

In [28]:
with open("rmse_RF.json", "w") as outfile:
    json.dump(rmse_dic_rf, outfile, indent=4)

with open("params_RF.json", "w") as outfile:
    json.dump(opt_params, outfile, indent=4)

### Comparison

In [29]:
with open('rmse_RF.json') as file:
    res_rf = json.load(file)
with open('params_RF.json') as file:
    params_rf = json.load(file)

with open('rmse_XGB.json') as file:
    res_gb = json.load(file)

with open('params_XGB.json') as file:
    params_gb = json.load(file)

In [30]:
opt_models = {}
for elem in tqdm(res_rf):
    diff = res_rf[elem] - res_gb[elem]
    if diff > 0:
        opt_params = params_gb[elem]
        model = XGBRegressor(n_estimators = opt_params[0], max_depth=opt_params[1], eta=opt_params[2], colsample_bytree=opt_params[3])
    else:
        opt_params = params_rf[elem]
        model = RandomForestRegressor(n_estimators=opt_params[0], max_depth=opt_params[1], min_samples_leaf=opt_params[2], min_samples_split=opt_params[3], n_jobs=-1)

    X = Utils.make_X(df, elem)
    y = globals()[name]
    model.fit(X,y)

    opt_models[elem] = model  


FI5 = {}
for elem in opt_models:
    if isinstance(opt_models[elem], XGBRegressor):
        FI = list(zip(opt_models[elem].feature_importances_, opt_models[elem].get_booster().feature_names))
        FI5[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]], 'GB')
    else:
        FI = list(zip(opt_models[elem].feature_importances_, Utils.make_X(df, elem).columns))
        FI5[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]], 'RF')

    
# with open('FeatureImportance.json', 'w') as file:
#     json.dump(FI5, file, indent=4)

  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [00:17<00:00,  1.66it/s]


In [31]:
with open('FeatureImportance.json', 'w') as file:
     json.dump(FI5, file, indent=4)

## Classification

In [ ]:
#%%capture

result = {}
for name in tqdm(var_names_cls):
    y = globals()[name]
    X = Utils.make_X(df, name)

    op, roc_auc, model = models.XGBCls(X, y)

    result[name] = (op, roc_auc, model)

In [14]:
roc_auc_dic_xgb = {}
opt_params = {}
for elem in result:
    roc_auc_dic_xgb[elem] = result[elem][1]
    opt_params[elem] = result[elem][0]

In [15]:
with open("roc_auc_XGB.json", "w") as outfile:
    json.dump(roc_auc_dic_xgb, outfile, indent=4)
with open("params_XGB_cls.json", "w") as outfile:
    json.dump(opt_params, outfile, indent=4)

### RF

In [17]:
result = {}
for name in tqdm(var_names_cls):
    y = globals()[name]
    X = Utils.make_X(df, name)

    op, roc_auc, model = models.RFReg(X, y, task='classification')

    result[name] = (op, roc_auc, model)

100%|██████████| 6/6 [03:19<00:00, 33.28s/it]


In [19]:
roc_auc_dic_rf = {}
opt_params = {}
for elem in result:
    roc_auc_dic_rf[elem] = result[elem][1]
    opt_params[elem] = result[elem][0]

In [20]:
with open("roc_auc_RF.json", "w") as outfile:
    json.dump(roc_auc_dic_rf, outfile, indent=4)

with open("params_RF_cls.json", "w") as outfile:
    json.dump(opt_params, outfile, indent=4)

### Comparison

In [21]:
with open('roc_auc_RF.json') as file:
    res_rf = json.load(file)
with open('params_RF_cls.json') as file:
    params_rf = json.load(file)

with open('roc_auc_XGB.json') as file:
    res_gb = json.load(file)

with open('params_XGB_cls.json') as file:
    params_gb = json.load(file)

In [24]:
opt_models = {}
for elem in tqdm(res_rf):
    diff = res_rf[elem] - res_gb[elem]
    if diff < 0:
        opt_params = params_gb[elem]
        model = XGBClassifier(n_estimators = opt_params[0], max_depth=opt_params[1], colsample_bytree=opt_params[2])
    else:
        opt_params = params_rf[elem]
        model = RandomForestClassifier(n_estimators=opt_params[0], max_depth=opt_params[1], min_samples_leaf=opt_params[2], min_samples_split=opt_params[3], n_jobs=-1)

    X = Utils.make_X(df, elem)
    y = globals()[name]
    model.fit(X,y)

    opt_models[elem] = model  


FI5 = {}
for elem in opt_models:
    if isinstance(opt_models[elem], XGBClassifier):
        FI = list(zip(opt_models[elem].feature_importances_, opt_models[elem].get_booster().feature_names))
        FI5[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]], 'GB')
    else:
        FI = list(zip(opt_models[elem].feature_importances_, Utils.make_X(df, elem).columns))
        FI5[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]], 'RF')

    
with open('FeatureImportance_CLS.json', 'w') as file:
        json.dump(FI5, file, indent=4)

100%|██████████| 6/6 [00:05<00:00,  1.09it/s]


In [25]:
FI5

{'O_12': ([('O_SUB', 0.32385159131298313),
   ('SUB2', 0.10272724571366676),
   ('n_sum', 0.053846706216948106),
   ('ADD2', 0.03916257677307681),
   ('ADD1', 0.026345658516828276)],
  'RF'),
 'O_23': ([('SUB3', 0.3005105257034302),
   ('DIFF_SUB32', 0.1838068664073944),
   ('O_SUB', 0.18066845834255219),
   ('SUB2', 0.16933616995811462),
   ('DIFF_DIV21', 0.11263544857501984)],
  'GB'),
 'O_ADD': ([('O_SUB', 0.2886989800331861),
   ('SUB3', 0.18133476711440022),
   ('SUB2', 0.07537745933152891),
   ('SUB1', 0.023852284219820802),
   ('MUL1', 0.018234815148016458)],
  'RF'),
 'O_DIV': ([('O_SUB', 0.2968141662316048),
   ('SUB3', 0.19470369796801312),
   ('SUB2', 0.061414650142464486),
   ('ADD2', 0.030881669427845468),
   ('SUB1', 0.020663527255670836)],
  'RF'),
 'O_MUL': ([('O_SUB', 0.4333386719226837),
   ('SUB3', 0.29542505741119385),
   ('ADD2', 0.1271250694990158),
   ('DIFF_ACC_SUB32', 0.03764766827225685),
   ('ADD1', 0.0373564176261425)],
  'GB'),
 'O_SUB': ([('O_MUL', 0.13120

In [21]:
opt_models

{'O_ADD': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=0.6, gamma=0, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.300000012, max_delta_step=0, max_depth=2,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=100, n_jobs=12, num_parallel_tree=1,
               objective='multi:softprob', random_state=0, reg_alpha=0,
               reg_lambda=1, scale_pos_weight=None, subsample=1,
               tree_method='exact', validate_parameters=1, verbosity=None),
 'O_DIV': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=0.6, gamma=0, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.300000012, max_delta_step=0, max_depth=2,
               min_child_weight=1, missing=nan, monoton